#### Imports

In [1]:
from environments import build_blue_agent, build_red_agent, sample

import ray
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
import logging
logging.disable(logging.WARNING)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
logger = logging.getLogger(__name__)

/home/garrett/miniconda3/envs/cyborg/lib/python3.8/site-packages/tensorflow/python/framework/dtypes.py:513: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.object,
/home/garrett/miniconda3/envs/cyborg/lib/python3.8/site-packages/tensorflow/python/framework/dtypes.py:521: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.bool,
/home/garrett/miniconda3/envs/cyborg/lib/python3.8/site-packages/tensorflow/python/fra

#### Verify GPU

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

#### Train Dedicated Agents

In [ ]:
# Start Ray
ray.init(ignore_reinit_error=True, log_to_driver=False)

red = build_red_agent(dedicated=True)

red_scores = [[],[],[],[],[]]

print("+--------------------+")
print("| Red Training Start |")
print("+--------------------+")

for g in range(1,5):
    path_file = open(f"./policies/red_dedicated_pool/dedicated_red_0/checkpoint_path", "r")
    red_restore_path = path_file.read()
    path_file.close()
    red.restore(red_restore_path)
    
    print()
    print("+--------------------+")
    print(f"| Red Policy {g} Start |")
    print("+--------------------+")
    print()

    red_max = 0

    for b in range(1, 101):
        result = red.train()
        red_score = result["sampler_results"]["episode_reward_mean"]
        print(f'Red Score for Batch {b}: {red_score:0.2f}')
        red_scores[g].append(red_score)
        if red_score > red_max:
            red_max = red_score
            checkpoint_path = red.save(checkpoint_dir=f"./policies/red_dedicated_pool/dedicated_red_{g}")
            path_file = open(f"./policies/red_dedicated_pool/dedicated_red_{g}/checkpoint_path", "w")
            path_file.write(checkpoint_path)
            path_file.close()
    print(checkpoint_path)

In [ ]:
# find dedicated agent that achieved the best score
r_max = 0
r_id = 0
for g in range(1,5):
    if max(red_scores[g]) > r_max:
        r_max = max(red_scores[g])
        r_id = g
print()
print(f'Dedicated Red Agent {r_id} reached the highest score.')

path_file = open(f"./policies/red_pool/dedicated_red_{r_id}/checkpoint_path", "r")
dedicated_red_path = path_file.read()
path_file.close()
path_file = open(f"./policies/dedicated_red_policy", "w")
path_file.write(dedicated_red_path)
path_file.close()

ray.shutdown()

In [ ]:
# Start Ray
ray.init(ignore_reinit_error=True, log_to_driver=False)

blue = build_blue_agent(dedicated=True)

blue_scores = [[],[],[],[],[]]

print()
print("+---------------------+")
print("| Blue Training Start |")
print("+---------------------+")
print()

for g in range(1,5):
    path_file = open(f"./policies/blue_dedicated_pool/dedicated_blue_0/checkpoint_path", "r")
    blue_restore_path = path_file.read()
    path_file.close()
    blue.restore(blue_restore_path)

    print()
    print("+---------------------+")
    print(f"| Blue Policy {g} Start |")
    print("+---------------------+")
    print()

    for b in range(1, 101):
        result = blue.train()
        blue_score = -result["sampler_results"]["episode_reward_mean"]
        print(f'Blue Score for Batch {b}: {blue_score:0.2f}')
        blue_scores[g].append(blue_score)
    checkpoint_path = blue.save(checkpoint_dir=f"./policies/blue_dedicated_pool/dedicated_blue_{g}")
    path_file = open(f"./policies/blue_dedicated_pool/dedicated_blue_{g}/checkpoint_path", "w")
    path_file.write(checkpoint_path)
    path_file.close()

In [ ]:
# find dedicated agents that achieved the best scores
b_min = float('inf')
b_id = 0

for g in range(1,5):
    if min(blue_scores[g]) < b_min:
        b_min = min(blue_scores[g])
        b_id = g

print()
print(f'Dedicated Blue Agent {b_id} reached the lowest score.')

path_file = open(f"./policies/blue_pool/dedicated_blue_{b_id}/checkpoint_path", "r")
dedicated_blue_path = path_file.read()
path_file.close()
path_file = open(f"./policies/dedicated_blue_policy", "w")
path_file.write(dedicated_blue_path)
path_file.close()

ray.shutdown()

#### Training Graphs

In [ ]:
# Scores During Training
id_plot = [id for id in range(1,101)]
for g in range(1,5):
    print(f'Dedicated Red Agent {g}')
    data_plot = pd.DataFrame({"Iteration":id_plot, "Training Score":red_scores[g]})
    plt.figure()
    sns.lineplot(x = "Iteration", y = "Training Score", data=data_plot, color='red')
    plt.show()


In [ ]:
id_plot = [id for id in range(1,101)]
for g in range(1,5):
    print(f'Dedicated Blue Agent {g}')
    data_plot = pd.DataFrame({"Iteration":id_plot, "Training Score":blue_scores[g]})
    plt.figure()
    sns.lineplot(x = "Iteration", y = "Training Score", data=data_plot, color='blue')
    plt.show()

#### Sample Game Between Competitive Policies

In [3]:
ray.init(ignore_reinit_error=True, log_to_driver=False)

red = build_red_agent(workers=1, fresh=False)
blue = build_blue_agent(workers=1, fresh=False)

In [ ]:
path_file = open(f"./policies/competitive_blue_policy", "r")
blue_restore_path = path_file.read()
path_file.close()
blue.restore(blue_restore_path)

path_file = open(f"./policies/competitive_red_policy", "r")
red_restore_path = path_file.read()
path_file.close()
red.restore(red_restore_path)

sample(red, blue, verbose=True, show_policy=True)

#### Sample Game between Competitive Blue and Dedicated Red

In [ ]:
red = build_red_agent(dedicated=True, workers=1, fresh=False)
blue = build_blue_agent(workers=1, fresh=False)

In [ ]:
path_file = open(f"./policies/competitive_blue_policy", "r")
blue_restore_path = path_file.read()
path_file.close()
blue.restore(blue_restore_path)

path_file = open(f"./policies/dedicated_red_policy", "r")
red_restore_path = path_file.read()
path_file.close()
red.restore(red_restore_path)

sample(red, blue, verbose=True, show_policy=True)

#### Sample Game between Dedicated Blue and Competitive Red

In [3]:
red = build_red_agent(workers=1, fresh=False)
blue = build_blue_agent(dedicated=True, workers=1, fresh=False)

In [ ]:
path_file = open(f"./policies/dedicated_blue_policy", "r")
blue_restore_path = path_file.read()
path_file.close()
blue.restore(blue_restore_path)

path_file = open(f"./policies/competitive_red_policy", "r")
red_restore_path = path_file.read()
path_file.close()
red.restore(red_restore_path)

sample(red, blue, verbose=True, show_policy=True)